In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from tensorflow.math import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Reshape X_train to include a channel dimension
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)  # Now X_train has the correct shape

In [7]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
checkpoint = keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')  # Change file extension to '.keras'
lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 10 ** (epoch / 20))

In [ ]:
history = model.fit(datagen.flow(X_train, Y_train, batch_size=32),
                    epochs=20,
                    validation_data=(X_test, Y_test),
                    callbacks=[checkpoint, lr_scheduler])

Epoch 1/20
   1/1875 ━━━━━━━━━━━━━━━━━━━━ 42:47 1s/step - accuracy: 0.0000e+00 - loss: 2.3306

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 35s 18ms/step - accuracy: 0.7622 - loss: 0.7546 - val_accuracy: 0.9503 - val_loss: 0.1506 - learning_rate: 0.0010
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.9305 - loss: 0.2225 - val_accuracy: 0.9626 - val_loss: 0.1239 - learning_rate: 0.0011
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - accuracy: 0.9425 - loss: 0.1846 - val_accuracy: 0.9666 - val_loss: 0.1051 - learning_rate: 0.0013
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.9493 - loss: 0.1644 - val_accuracy: 0.9667 - val_loss: 0.1009 - learning_rate: 0.0014
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.9551 - loss: 0.1484 - val_accuracy: 0.9763 - val_loss: 0.0762 - learning_rate: 0.0016
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.9563 - loss: 0.1424 - val_accuracy: 0.9722 - val_loss: 0.0930 - learning_rate: 0.0018
Epoch 7/20
1735/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9558 -

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test accuracy: {accuracy}')

In [ ]:
Y_pred = model.predict(X_test)
Y_pred_labels = np.argmax(Y_pred, axis=1)

In [ ]:
print(classification_report(Y_test, Y_pred_labels, target_names=[str(i) for i in range(10)]))

In [ ]:
conf_mat = confusion_matrix(Y_test, Y_pred_labels)
plt.figure(figsize=(15, 7))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()

In [ ]:
input_image_path = 'images.jpg'
input_image = cv2.imread(input_image_path)

In [ ]:
if input_image is not None:
    cv2_imshow(input_image)

In [ ]:
grayscale = cv2.cvtColor(input_image, cv2.COLOR_RGB2GRAY)
input_image_resize = cv2.resize(grayscale, (28, 28))
input_image_resize = input_image_resize / 255.0

In [ ]:
image_reshaped = np.reshape(input_image_resize, [1, 28, 28])
input_prediction = model.predict(image_reshaped)
input_pred_label = np.argmax(input_prediction)
print(f'Predicted Label: {input_pred_label}')